Notebook Settings
=================

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run ../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'

REPO_ROOT = "/home/leon/models/NeuroFlame"
pal = sns.color_palette("tab10")
```

Helpers
=======

``` ipython
def get_theta(a, b, GM=0, IF_NORM=0):

    u, v = a, b

    if GM:          
        v = b - np.dot(b, a) / np.dot(a, a) * a

    if IF_NORM:
        u = a / np.linalg.norm(a)
        v = b / np.linalg.norm(b)

    return np.arctan2(v, u)
```

``` ipython
def get_idx(ksi, rank=2):
    ksi = ksi.cpu().detach().numpy()      
    theta = get_theta(ksi[0], ksi[rank], GM=0, IF_NORM=0)

    return theta.argsort()
```

``` ipython
def get_overlap(ksi, rates):
    return rates @ ksi.T / rates.shape[-1]  
```

``` ipython
import scipy.stats as stats

def plot_smooth(data, ax, color):
    mean = data.mean(axis=0)  
    ci = smooth.std(axis=0, ddof=1) * 1.96

    # Plot
    ax.plot(mean, color=color)
    ax.fill_between(range(data.shape[1]), mean - ci, mean + ci, alpha=0.25, color=color)

```

``` ipython
def convert_seconds(seconds):
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return h, m, s
```

Imports
=======

``` ipython
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader
```

``` ipython
import sys
sys.path.insert(0, '../')

import pandas as pd
import torch.nn as nn
from time import perf_counter  
from scipy.stats import circmean

from src.network import Network
from src.plot_utils import plot_con
from src.decode import decode_bump, circcvl
```

Spectrum
========

Sparse Matrix Spectrum
----------------------

``` ipython
from src.connectivity import Connectivity
```

``` ipython
Wij = Connectivity(1000, 1000, 100)('sparse', 'None', kappa=1.0, sigma=0, phase=0)
```

``` ipython
eigenvalues, eigenvectors = torch.linalg.eig(1.0 * Wij)
max_real_index = torch.argmax(eigenvalues.real)
lead_eig_vec = eigenvectors[:, max_real_index]
```

``` ipython
plt.plot(eigenvalues.real.cpu().numpy(), eigenvalues.imag.cpu().numpy(), 'o')
plt.xlabel('Real')
plt.ylabel('Im')
# plt.xlim([-10, 10])
plt.show()
```

Orthogonal LR
-------------

``` ipython
from src.lr_utils import get_ortho_pertu, gen_ortho_vec
```

``` ipython
# u, v = get_ortho_pertu(lead_eig_vec, 0.0, dtype=torch.float32, device='cuda')
u, v = gen_ortho_vec(lead_eig_vec.real, 0.5, dtype=torch.float32, device='cuda')
```

``` ipython
print(u.shape, v.shape)
```

``` ipython
Lij = u.unsqueeze(-1) @ u.unsqueeze(-1).T
Lij = Lij + v.unsqueeze(-1) @ v.unsqueeze(-1).T
print(Lij.shape)
Jij = Wij + Lij * torch.sqrt(torch.tensor(100.0 / 1000.0, dtype=torch.float32, device='cuda'))
```

K/N (1 + 1/sqrt(K)) = K/N + sqrt(K) / N

``` example
torch.Size([1000, 1000])
```

``` ipython
eig_val, eig_vec = torch.linalg.eig(1.0 * Lij)
max_real_idx = torch.argmax(eig_val.real)
lead_eig_vec = eig_vec[:, max_real_idx]
```

``` ipython
plt.plot(eig_val.real.cpu().numpy(), eig_val.imag.cpu().numpy(), 'ro')
plt.xlabel('Real')
plt.ylabel('Im')
plt.show()
```

``` ipython
eig_val, eig_vec = torch.linalg.eig(1.0 * Jij)
max_real_idx = torch.argmax(eig_val.real)
lead_eig_vec = eig_vec[:, max_real_idx]
```

``` ipython
plt.plot(eigenvalues.real.cpu().numpy(), eigenvalues.imag.cpu().numpy(), 'bo')
plt.plot(eig_val.real.cpu().numpy(), eig_val.imag.cpu().numpy(), 'ro')
plt.xlabel('Real')
plt.ylabel('Im')
plt.show()
```

Random LR
---------

``` ipython
from torch.distributions import MultivariateNormal
```

``` ipython
lr_mean = [0.0, 0.0]
lr_cov = [[1.0, 0.0],[0.0, 1.0]]

mean_ = torch.tensor(lr_mean, dtype=torch.float32, device='cuda')
cov_ = torch.tensor(lr_cov, dtype=torch.float32, device='cuda')

mv_normal = MultivariateNormal(mean_, cov_)
ksi = mv_normal.sample((1000,)).T
print(ksi.shape)
```

``` ipython
# Lij = ksi[0].unsqueeze(-1) @ ksi[1].unsqueeze(-1).T
Lij = ksi[0].unsqueeze(-1) @ ksi[0].unsqueeze(-1).T
Lij = Lij + ksi[1].unsqueeze(-1) @ ksi[1].unsqueeze(-1).T
print(Lij.shape)
Gij = Wij + Lij * torch.sqrt(torch.tensor(100.0 / 1000.0, dtype=torch.float32, device='cuda'))
```

``` ipython
eig_val, eig_vec = torch.linalg.eig(1.0 * Lij)
max_real_idx = torch.argmax(eig_val.real)
lead_eig_vec = eig_vec[:, max_real_idx]
```

``` ipython
plt.plot(eig_val.real.cpu().numpy(), eig_val.imag.cpu().numpy(), 'ro')
plt.xlabel('Real')
plt.ylabel('Im')
plt.show()
```

``` ipython
eig_val, eig_vec = torch.linalg.eig(1.0 * Gij)
max_real_idx = torch.argmax(eig_val.real)
lead_eig_vec = eig_vec[:, max_real_idx]
```

``` ipython
plt.plot(eigenvalues.real.cpu().numpy(), eigenvalues.imag.cpu().numpy(), 'bo')
plt.plot(eig_val.real.cpu().numpy(), eig_val.imag.cpu().numpy(), 'ro')
plt.xlabel('Real')
plt.ylabel('Im')
# plt.xlim([-10, 10])
plt.show()
```

``` ipython
```

Model
=====

``` ipython
u = torch.randn(model.Na[0], dtype=torch.float32, device='cuda')
u = u / u.std()
```

Single Trial
------------

### Model

``` ipython
from src.network import Network
```

``` ipython
REPO_ROOT = "/home/leon/models/NeuroFlame"
conf_name = "config_EI.yml"
```

``` ipython
start = perf_counter()
model = Network(conf_name, REPO_ROOT, VERBOSE=1, DEVICE='cuda', LIVE_FF_UPDATE=1, TASK='dual_rand', seed=1, PROBA_TYPE=['','','',''], N_BATCH=100, DURATION=30)
```

``` ipython
Wij = model.Wab_T.clone()

# eigenvalues, eigenvectors = torch.linalg.eig(Wij.T)
eigenvalues, eigenvectors = torch.linalg.eig(Wij[model.slices[0], model.slices[0]].T)
max_real_index = torch.argmax(eigenvalues.real)
lead_eig_vec = eigenvectors[:, max_real_index]
```

``` ipython
plt.plot(eigenvalues.real.cpu().numpy(), eigenvalues.imag.cpu().numpy(), 'o')
plt.xlabel('Real')
plt.ylabel('Im')
# plt.xlim([-10, 10])
plt.show()
```

### ortho LR

``` ipython
from src.lr_utils import gen_v_cov
```

``` ipython
cov = torch.tensor(0.0, dtype=torch.float32, device='cuda')
```

``` ipython
v = gen_v_cov(u, cov, dtype=torch.float32, device='cuda')
```

``` ipython
# u, v = gen_ortho_vec(lead_eig_vec.real, 0.0, dtype=torch.float32, device='cuda')
print(u.shape, v.shape)
```

K/N \* 1 / sqrt(K) \* Jij + Lij / N = sqrt(K) Jij / N + Lij / N

``` ipython
Lij = u.unsqueeze(-1) @ u.unsqueeze(-1).T
Lij = Lij + v.unsqueeze(-1) @ v.unsqueeze(-1).T
Lij = Lij / model.Ka[0]
print(Lij.shape)
```

``` ipython
ksi = torch.stack((u, v))
print(torch.cov(ksi))
print(ksi.shape)
```

``` ipython
model.Wab_T = Wij.clone()
model.Wab_T[model.slices[0], model.slices[0]] = (Wij[model.slices[0], model.slices[0]] + Lij).T.clamp_(min=0, max=1.0 * model.Jab[0, 0])
```

### Dynamics

``` ipython
model.PHI0[0] = ksi[0]
model.PHI0[1] = ksi[0]
model.PHI0[1] = ksi[1]
model.PHI0[2] = ksi[1]

rates = model()
end = perf_counter()
print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))

Ne = model.Na[0].detach().cpu().numpy()
N = model.N_NEURON

print('rates', rates.shape)
```

``` ipython
overlap = get_overlap(ksi, rates)
print(overlap.shape)

idx = get_idx(ksi, rank=1)
rates_ordered = rates[..., idx].cpu().numpy()

m0, m1, phi = decode_bump(rates_ordered, axis=-1)
print(m0.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=(2*width, height))

r_max = 5

ax[0].imshow(rates[0].T.cpu().numpy(), aspect='auto', cmap='jet', vmin=0, vmax=r_max, origin='lower')
ax[0].set_ylabel('Neuron #')
ax[0].set_xlabel('Step')

ax[1].imshow(rates_ordered[0].T, aspect='auto', cmap='jet', vmin=0, vmax=r_max)
ax[1].set_ylabel('Pref. Location (°)')
ax[1].set_xlabel('Step')
ax[1].set_yticks(np.linspace(0, Ne, 5), np.linspace(360, 0, 5).astype(int))
# ax[0][1].colorbar()

plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=(2*width, height))

ax[0].plot((m1.T/m0.T))
ax[0].set_xlabel('Step')
ax[0].set_ylabel('$\mathcal{F}_1$')

ax[1].plot((phi.T * 180 / np.pi))
ax[1].set_yticks(np.linspace(0, 360, 5).astype(int), np.linspace(0, 360, 5).astype(int))
ax[1].set_xlabel('Step')
ax[1].set_ylabel('Phase (°)')
plt.show()
```

``` ipython
x = m1[:, -1]/ m0[:, -1] * np.cos(phi[:, -1])
y = m1[:, -1] / m0[:, -1] * np.sin(phi[:, -1])

fig, ax = plt.subplots(figsize=(height, height))
ax.plot(x, y, 'o')
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
plt.show()
```

``` ipython
```